# Домашнє завдання до теми "Розширені методи обробки даних у Pandas"

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [1]:
import pandas as pd
data_path = r"C:\Users\limon.DESKTOP-D4OD6GD\OneDrive\Desktop\Навчання\python\supermarket_sales.csv"
df = pd.read_csv(data_path)
type(df)

pandas.core.frame.DataFrame

1. Відсортуйте дані за спаданням ціни товара ('Unit price') і виведіть 5 перших рядочків на екран.

In [15]:
df.sort_values(by='Unit price', ascending=False).head()


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Hour
983,148-41-7930,C,Naypyitaw,Normal,Male,Health and beauty,99.96,7,34.986,734.706,1/23/2019,10:33,Cash,699.72,4.761905,34.986,6.1,10
122,219-22-9386,B,Mandalay,Member,Male,Sports and travel,99.96,9,44.982,944.622,3/9/2019,17:26,Credit card,899.64,4.761905,44.982,4.2,17
930,641-62-7288,B,Mandalay,Normal,Male,Home and lifestyle,99.92,6,29.976,629.496,3/24/2019,13:33,Ewallet,599.52,4.761905,29.976,7.1,13
494,437-53-3084,B,Mandalay,Normal,Male,Fashion accessories,99.89,2,9.989,209.769,2/26/2019,11:48,Ewallet,199.78,4.761905,9.989,7.1,11
283,667-92-0055,A,Yangon,Member,Male,Health and beauty,99.83,6,29.949,628.929,3/4/2019,15:02,Ewallet,598.98,4.761905,29.949,8.5,15


2. Знайдіть інвойс (рядок в даних) в категорії товарів `Electronic accessories`, в якому був куплений товар з найбільшою кількістю і найбільшою ціною. Виведіть цей рядок даних на екран.

In [17]:
df[df['Product line'] == 'Electronic accessories'] \
  .sort_values(['Quantity', 'Unit price'], ascending=[False, False]) \
  .head(1)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Hour
105,704-48-3927,A,Yangon,Member,Male,Electronic accessories,88.67,10,44.335,931.035,1/12/2019,14:50,Ewallet,886.7,4.761905,44.335,7.3,14


3.1. Створіть нову колонку в даних `Rating Category`, яка визначає категорію рейтингу (`Rating`): високий, середній чи низький за наступним правилом:
- якщо рейтинг вищий або рівний 8 - то він високий (`High`)
- якщо рейтинг від 6 (включно) до 8 (не включно) - то він середній (`Medium`)
- інакше - низький (`Low`).

In [20]:
def rating_category(rating):
    if rating >= 8:
        return 'High'
    elif rating >= 6:
        return 'Medium'
    else:
        return 'Low'
df['Rating Category'] = df['Rating'].apply(rating_category)
df[['Rating', 'Rating Category']].head(10)


,Rating,Rating Category
0,9.1,High
1,9.6,High
2,7.4,Medium
3,8.4,High
4,5.3,Low
5,4.1,Low
6,5.8,Low
7,8.0,High
8,7.2,Medium
9,5.9,Low


3.2. Виведіть, який відсоток даних становить кожне зі значень категорій рейтингу.

In [9]:
df['Rating'].value_counts(normalize=True) * 100


Rating
6.0     2.6
6.6     2.4
4.2     2.2
9.5     2.2
6.5     2.1
       ... 
4.0     1.1
5.3     1.1
8.3     1.1
4.6     0.8
10.0    0.5
Name: proportion, Length: 61, dtype: float64

4.1. Запустіть код нижче аби створити змінну `time`. Ця змінна містить час в форматі "година:хвилина". Напишіть функцію `get_hour`, яка буде зі змінної формату як `time` дістати лише години і конвертувати їх у тип `int`.

Запустіть написану функцію на змінній `time` та перевірте правильність роботи функції.

In [10]:
time = '11:28'

In [12]:
def get_hour(time):
    return int(time.split(':')[0])
get_hour(time)

11

4.2. З допомогою функції `get_hour` і метода `DataFrame.apply()` створіть нову колонку `Hour` в нашому наборі даних, яка буде містити години здійснення покупки, обчислені на основі часу здійснення покупки `Time`.

In [13]:
df['Hour'] = df['Time'].apply(get_hour)


4.3. Виведіть кількість значень за кожною годиною покупки і дайте відповідь "О яких годинах найбільше здійснюють покупок покупці і чому на Ваш погляд саме о цих годинах? Чи це слогічно, що найбільше покупок саме в ці години?".

In [22]:
df['Hour'].value_counts().sort_index()


Hour
10    101
11     90
12     89
13    103
14     83
15    102
16     77
17     74
18     93
19    113
20     75
Name: count, dtype: int64

Найбільша кількість покупок здійснюється о 19:00, що, ймовірно, пов’язано із завершенням робочого дня, коли люди заходять у супермаркет дорогою додому. Також високий рівень покупок спостерігається в обідні години (близько 13:00) та в другій половині дня. Це логічно, оскільки в ці проміжки часу потік клієнтів є найбільшим.

5. Виведіть середнє значення чеку (`Total`) для покупок в містах, наявних в даних, за допомогою функції `groupby`.

In [23]:
df.groupby('City')['Total'].mean()


City
Mandalay     319.872506
Naypyitaw    337.099715
Yangon       312.354031
Name: Total, dtype: float64

6. Виведіть кількість покупок, середнє, мінімальне та максимальне значення для загальної суми чеку (Total) в містах, наявних в даних, за допомогою функції groupby.
Зробіть висновки з цих показників та дайте рекомендації для мережі супермаркетів.

In [26]:
df.groupby('City')['Total'].agg(['count', 'mean', 'min', 'max']).round(3)


,count,mean,min,max
City,,,,
Mandalay,332,319.873,18.638,1022.49
Naypyitaw,328,337.100,10.678,1042.65
Yangon,340,312.354,12.694,1039.29


7. Ми внесли деякі зміни в наш набір даних, тож, давайте запишемо результат у файл. Запишіть оновлену версію даних у файл `supermarket_sales_enriched.csv`. Зверніть увагу, аби не записати нічого зайвого, наприклад, індекси рядків нам в файлі не потрібні, оскільки вони стандартні.

In [28]:
df.to_csv(
    "C:/Users/limon.DESKTOP-D4OD6GD/OneDrive/Desktop/Навчання/python/supermarket_sales_enriched.csv",
    index=False
)
